<a href="https://colab.research.google.com/github/mendesvi/COMPILADORES/blob/main/P4_Semantic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Quarto Projeto: Analisador Semântico

Depois que a árvore de sintaxe abstrata (do inglês, *Abstract Syntax Tree* - AST) é construída, uma análise adicional pode ser feita avaliando os atributos dos nós da árvore para reunir as informações semânticas necessárias do código-fonte que não são facilmente detectadas durante a análise sintática. Isso geralmente inclui a verificação de tipos, a construção da tabela de símbolos para garantir que uma variável seja declarada antes do uso e a decoração da AST para prepará-la para a próxima fase de compilação.

## Sistema de Tipos

Primeiro, você precisará definir objetos que representem os diferentes tipos de dados básicos e armazenar informações sobre suas características.

Vamos definir classes que representam tipos. Existe uma classe geral usada para representar todos os tipos. Cada tipo básico é então uma única instância da classe de tipo.

```python
class uCType:
      pass

int_type = uCType("int", ...)
char_type = uCType("char", ...)
```

O conteúdo da classe de tipo depende inteiramente de você. No entanto, você precisará codificar minimamente algumas informações sobre quais operadores são suportados (+, -, *, etc.) e valores padrão.

Depois de definir os tipos básicos, você precisará certificar-se de que eles sejam registrados com qualquer tabela de símbolos ou código que verifique os nomes dos tipos.

In [ ]:
class Type:
    """
    Class that represents a type in the language. Basic
    Types are declared as singleton instances of this type.
    """

    def __init__(self, name):
        self.typename = name

    def __repr__(self):
        return self.typename

class uCType(Type):

    def __init__(self, name, binary_ops=set(), unary_ops=set(), rel_ops=set(), assign_ops=set()):
        super().__init__(name)
        self.binary_ops = binary_ops
        self.unary_ops = unary_ops
        self.rel_ops = rel_ops
        self.assign_ops = assign_ops

# Create specific instances of basic types. You will need to add
# appropriate arguments depending on your definition of Type
IntType = uCType(
    "int",
    unary_ops  = {"-", "+"},
    binary_ops = {"+", "-", "*", "/", "%"},
    rel_ops    = {"==", "!=", "<", ">", "<=", ">="},
    assign_ops = {"="}
)

CharType = uCType(
    "char",
    rel_ops    = {"==", "!=", "&&", "||"},
    assign_ops = {"="}
)

StringType = uCType(
    "string",
    binary_ops = {"+"},
    rel_ops    = {"==", "!="},
)

BoolType = uCType(
    "bool",
    unary_ops  = {"!"},
    rel_ops    = {"==", "!=", "&&", "||"},
    assign_ops = {"="}
)

VoidType = uCType("void")


# Array & Function types need to be instantiated for each declaration
def represent_array(obj):
    def _repr(obj):
        if isinstance(obj.type, ArrayType):
            return '['+(str(obj.size) if obj.size is not None else '')+']'+_repr(obj.type)
        else:
            return '['+(str(obj.size) if obj.size is not None else '')+']'

    if isinstance(obj, ArrayType):
        return obj.typename+_repr(obj)
    
class ArrayType(uCType):
    
    def __init__(self, element_type, size=None):
        """
        type: Any of the uCTypes can be used as the array's type. This
              means that there's support for nested types, like matrices.
        size: Integer with the length of the array.
        """
        super().__init__(element_type.typename, rel_ops={"==", "!="}, assign_ops = {"="})
        self.type = element_type
        self.size = size

    def __repr__(self):
        return represent_array(self)

    def __eq__(self, other):
        return isinstance(other, ArrayType) and self.size == other.size and self.type == other.type

class FunctionType(uCType):
    
    def __init__(self, return_type, param_list=None):
        """
        type: Any of the uCTypes can be used as the function's type.
        params: List of the uCTypes of all parameters of the function.
        """
        super().__init__(return_type.typename)
        self.type = return_type
        self.params = param_list

    def __repr__(self):
        return self.typename+' ('+', '.join(map(str, self.params))+')' 

    def __eq__(self, other):
        return isinstance(other, FunctionType) and self.type == other.type and self.params == other.params

Em seu código de verificação de tipos, você precisará fazer referência aos objetos de tipo acima. Pense em como você vai querer acessá-los.

## Tabela de Símbolos

Você precisará definir uma tabela de símbolos que acompanhe os identificadores declarados anteriormente. A tabela de símbolos será consultada sempre que o compilador precisar consultar informações sobre declarações de variáveis e constantes.

In [ ]:
class SymbolTable(dict):
    """ Class representing a symbol table. It should provide functionality
        for adding and looking up nodes associated with identifiers.
    """
    def __init__(self, parent=None):
        super().__init__()
        self.parent = parent

    def add(self, name, value):
        self[name] = value

    def lookup(self, name):
        if self.parent is not None:
            return self.get(name, self.parent.lookup(name))
        else:
            return self.get(name, None)
    
    def root(self):
        if self.parent is not None:
            return self.parent.root()
        else:
            return self

Observe que a análise semântica terá que gerenciar a tabela de símbolos para lidar com os múltiplos escopos do programa.

## Visitando a AST

O padrão de visitante é frequentemente usado no compilador para percorrer estruturas de dados que representam os programas, seja árvore de sintaxe ou qualquer outra representação intermediária. Para isso, fornecemos a seguinte classe `NodeVisitor` para permitir que você visite a AST. Essa classe foi modelada com base nas instalações de visitação AST do próprio Python (o módulo AST do Python 3).

In [ ]:
class NodeVisitor:
    """ A base NodeVisitor class for visiting uc_ast nodes.
        Subclass it and define your own visit_XXX methods, where
        XXX is the class name you want to visit with these
        methods.
    """

    _method_cache = None

    def visit(self, node):
        """ Visit a node.
        """

        if self._method_cache is None:
            self._method_cache = {}

        visitor = self._method_cache.get(node.__class__.__name__, None)
        if visitor is None:
            method = 'visit_' + node.__class__.__name__
            visitor = getattr(self, method, self.generic_visit)
            self._method_cache[node.__class__.__name__] = visitor

        return visitor(node)

    def generic_visit(self, node):
        """ Called if no explicit visitor function exists for a
            node. Implements preorder visiting of the node.
        """
        for _, child in node.children():
            self.visit(child)

Por exemplo, um pequeno visitante para constante pode ser implementado desta forma:

In [ ]:
class ConstantVisitor(NodeVisitor):
    def __init__(self):
        self.values = []

    def visit_Constant(self, node):
        self.values.append(node.value)

Esse visitante criaria uma lista de valores de todos os nós constantes encontrados abaixo de um determinado nó. Para usá-lo, basta instanciar o visitante e chamar seu método `visit` no nó de sua escolha:

```python
cv = ConstantVisitor()
cv.visit(node)
```

Observe que:

- O método `generic_visit()` será chamado para nós AST para os quais nenhum método `visit_XXX` foi definido.
- Os filhos de nós para os quais um método `visit_XXX` foi definido não serão visitados - se você precisar disso, chame `generic_visit()` no nó.
- O método `generic_visit()` pode ser implementado de forma mais eficiente definindo os nós AST como objeto iterável usando o método `__iter__()` do Python. Sinta-se à vontade para otimizar.

## Erros Padronizados

Para verificar erros semânticos e imprimir sua descrição, deve-se usar o seguinte método:

```python
    def _assert_semantic(self, condition, msg_code, coord, name="", ltype="", rtype=""):
        """Check condition, if false print selected error message and exit"""
        error_msgs = {
             1: f"'{name}' is not defined",
             2: f"subscript must be of type 'int', not type '{ltype}'",
             3: f"Expression must be of type 'bool', not type '{ltype}'",
             4: f"Cannot assign type '{rtype}' to type '{ltype}'",
             5: f"Assignment operator '{name}' is not supported by type '{ltype}'",
             6: f"Binary operator '{name}' does not have matching LHS/RHS types",
             7: f"Binary operator '{name}' is not supported by type '{ltype}'",
             8: "Break statement must be inside a loop",
             9: f"Array has incomplete element type '{ltype}'",
            10: f"Size mismatch on '{name}' initialization",
            11: f"'{name}' initialization type mismatch",
            12: f"'{name}' initialization must be a single element",
            13: "Lists have different sizes",
            14: "List & variable have different sizes",
            15: f"Variable declared as array of functions of type '{ltype}'",
            16: f"'{name}' is not a function",
            17: f"no. arguments to call '{name}' function mismatch",
            18: f"Type mismatch with parameter '{name}'",
            19: f"The condition expression must be of type(bool), not type '{ltype}'",
            20: "Expression must be a constant",
            21: "Expression is not of basic type",
            22: f"'{name}' does not reference a variable of basic type",
            23: f"'{name}' is not a variable",
            24: f"Return of type '{ltype}' is incompatible with type '{rtype}' function definition",
            25: f"Name '{name}' is already defined in this scope",
            26: f"Unary operator '{name}' is not supported by type '{ltype}'",
            27: f"Conflicting declarations for function '{name}'",
            28: f"'{name}' initialization must be a list or string literal",
            29: "Lists have different types",
            30: "Subscripted value is not an array",
            31: "Expression must be a variable",
            32: "Expression is not assignable",
            33: f"Variable has incomplete type '{ltype}'",
            34: f"'{name}' initialization must be a list",
            35: "Undefined error",
        }
        if not condition:
            msg = error_msgs.get(msg_code)
            print("SemanticError: %s %s" % (msg, coord), file=sys.stdout)
            sys.exit(1)
```

Conforme mostrado no código da função, cada mensagem está associada a um código de erro específico. A mensagem de erro é impressa de acordo com as coordenadas fornecidas e os argumentos opcionais. O método `_assert_semantic` é usado para padronizar a saída da análise semântica e deve ser usado para passar nos testes automáticos. Alguns exemplos que mostram como usar o método são fornecidos na próxima seção.

## Implementando o Análisador Semântico

Por fim, você precisará escrever um código que percorra a AST, decore-a com informações adicionais e imponha um conjunto de regras semânticas, conforme explicado pelas diretrizes abaixo. Para percorrer a AST, use a classe `NodeVisitor`. Uma implementação inicial do analisador semântico é fornecida no código abaixo.

```python
class Visitor(NodeVisitor):
    '''
    Program visitor class. This class uses the visitor pattern. You need to define methods
    of the form visit_NodeName() for each kind of AST node that you want to process.
    '''
    def __init__(self):
        # Keep a reference to current symbol table
        self.symtab = None

        # Keep a reference to current function
        self.func = None

        # Add built-in type names (int, char, void)
        self.typemap = {
            "int": IntType,
            "char": CharType,
            "string": StringType,
            "bool": BoolType,
            "void": VoidType,
        }
        
    def visit_Assignment(self, node):
        # Visit the right side
        self.visit(node.rvalue)
        rtype = node.rvalue.attrs['uc_type']
        # Visit the left side
        self.visit(node.lvalue)        
        ltype = node.lvalue.attrs['uc_type']
        # Check that the assignment is allowed otherwise return a type error (code 4)
        self._assert_semantic(ltype == rtype, 4, node.coord, ltype=ltype, rtype=rtype)
        # Check that assign_ops is supported by the type or return an error (code 5)
        self._assert_semantic(node.op in ltype.assign_ops, 5, node.coord, name=node.op, ltype=ltype)
        # Assign the type of the left side to current node
        node.attrs['uc_type'] = ltype

    def visit_BinaryOp(self,node):
        # Visit the left and right expression
        self.visit(node.left)
        ltype = node.left.attrs['uc_type']
        self.visit(node.right)
        rtype = node.right.attrs['uc_type']
        # TODO: 
        # - Make sure left and right operands have the same type
        # - Make sure the operation is supported
        # - Assign the result type to current node
     
    def visit_Program(self, node):
        # Create a symbol table for the file scope and assign it to current node
        node.attrs['symtab'] = SymbolTable()
        # Set the reference to current symbol table to the new symbol table
        self.symtab = node.attrs['symtab']
        # Visit all of the global declarations
        for _decl in node.gdecls:
            self.visit(_decl)
```

**IMPORTANTE:** A AST que você criou anteriormente contém apenas informações (como tipos) em nós específicos. Além de encontrar os possíveis erros remanescentes do programa, a análise semântica deve ser usada para descobrir informações adicionais (como o tipo de todas as expressões) que serão úteis para a geração de código, a próxima fase de compilação. Este processo é geralmente chamado de "decorar a AST".

## Diretrizes

Além disso, fornecemos um conjunto de diretrizes que podem ser usadas para implementar cada função da análise semântica (verificação de tipo, verificação de definição, etc). Por favor, leia-as com atenção.

### Programa / Funções

1. Programa (`visit_Program`)

- Crie uma tabela de símbolos para o escopo do arquivo e atribua-a ao nó atual (`attrs['symtab']`)
- Ajuste a referência à tabela de símbolos atual para a nova tabela de símbolos
- Visite todas as declarações globais (`gdecls`)

2. Definição de função (`visit_FuncDef`)

- Inicialize uma lista de declarações que aparece dentro de laços e atribua-a ao nó atual (`attrs['loops']`)
- Visite a declaração da função (`decl`)
- Visite o tipo de retorno da função (`spec`)
- Salve a referência à função atual e atribua-a ao nó atual
- Visite o corpo da função (`body`)
- Restaure a referência à função atual

3. Lista de parâmetros (`visit_ParamList`)

- Crie uma tabela de símbolos para o escopo do protótipo da função e atribua-a ao nó atual (`attrs['symtab']`)
- Salve a referência à tabela de símbolos atual e atribua-a à nova tabela de símbolos
- Visite todos os parâmetros (`params`)
- Restaure a referência à tabela de símbolos anterior


### Declarações / Tipo

1. Declaração global (`visit_GlobalDecl`)

- Basta visitar cada declaração global (`decls`)

2. Declaração (`visit_Decl`)

- Primeiro, visite o tipo (`type`) para ajustar a lista de tipos para objetos `uCType`
- Então, pegue o nome (`name`) da função ou da variável e verifique se ela não está definida
    - Se o tipo for uma função, verifique se ela não está definida em nenhum escopo ou, se estiver, se a declaração atual e a anterior são compatívies (mesmo tipo de retorno, número e tipos de parâmetros), caso contrário, retorne um erro (código 27)
    - Se o tipo for uma variável, verifique se ela não está definida no escopo do bloco atual, caso contrário, retorne um erro (código 25)
    - Além disso, certifique-se de que o tipo da variável não seja `void` (`VoidType`) ou retorne um erro (código 33)
- Em seguida, insira seu identificador na tabela de símbolos no escopo do bloco atual
- Depois disso, atribua o identificador ao escopo do bloco atual (`attrs['scope']`)
- Por fim, copie o tipo para o identificador (`attrs['uc_type']`)
- Se o tipo for uma função, copie o escopo do protótipo da função para o identificador (`attrs['scope']`) e insira-o na tabela de símbolos no escopo do arquivo
- Verifique se um valor inicial (`init`) está definido
    - Se um valor inicial estiver definido, verifique se o tipo é uma variável, caso contrário, retorne um erro (código 23)
    - Em seguida, visite o valor inicial
        - Se a variável for do tipo arranjo, certifique-se de que o valor inicial seja uma lista (`InitList`) ou uma string literal (`StringType`) ou retorne um erro (código 28)
        - Em seguida, verifique se o valor inicial é uma lista (`InitList`) ou uma string literal (`StringType`)
            - Se o valor inicial for uma lista (`InitList`), certifique-se de que os elementos do arranjo e da lista sejam do mesmo tipo, caso contrário, retorne um erro de tipo (código 11)
            - Em seguida, verifique se o tamanho do arranjo está definido
                - Se o tamanho do arranjo estiver definido, certifique-se de que o tamanho do arranjo e da lista sejam idênticos ou retorne um erro (código 14)
                - Se o tamanho do arranjo não estiver definido, defina seu valor para o tamanho da lista
            - Se o valor inicial for uma string literal (`StringType`), certifique-se de que os elementos do arranjo não sejam do tipo arranjo ou retorne um erro (código 34)
            - Então, verifique se os elementos do arranjo são do tipo `char` (`CharType`) ou retorne um erro (código 11)
            - Em seguida, instancie um visitante constante (`ConstantVisitor`) e visite o valor inicial para criar uma lista de strings literais de todos os nós constantes
            - Depois disso, some o comprimento (excluindo aspas) de todas as strings literais da lista criada pelo visitante constante
            - Por fim, verifique se o tamanho do arranjo está definido
                - Se o tamanho do arranjo estiver definido, certifique-se de que o tamanho do arranjo e a soma dos comprimentos das strings literais sejam idênticos ou retorne um erro (código 10)
                - Se o tamanho do arranjo não estiver definido, defina seu valor como a soma dos comprimentos das strings literais
        - Se a variável for de um tipo básico, verifique se o valor inicial é uma lista (`InitList`)
            - Se o valor inicial for uma lista (`InitList`), certifique-se de que ela contém um único elemento ou retorne um erro (código 12)
        - Em seguida, verifique se a variável e o valor inicial são do mesmo tipo ou retorne um erro de tipo (código 4)
    - Se um valor inicial não estiver definido e o tipo for um arranjo, verifique se o tamanho do arranjo está definido ou retorne um erro (código 9)

3. Declaração de variável (`visit_VarDecl`)

- Visite o tipo (`type`) para ajustar a lista de tipos para objetos `uCType`
- Atribua esse tipo ao nó atual (`attrs['uc_type']`)

4. Declaração de arranjo (`visit_ArrayDecl`)

- Comece visitando o tipo (`type`) para ajustar a lista de tipos aos objetos `uCType`
- Certifique-se de que os elementos do arranjo não sejam do tipo função, caso contrário retorne um erro (código 15)
- Verifique se os elementos do arranjo são do tipo arranjo
    - Se eles forem do tipo arranjo, certifique-se de que seus tamanhos estejam definidos ou retorne um erro (código 9)
- Verifique se o tamanho do arranjo (`size`) está definido
    - Se o tamanho do arranjo estiver definido, certifique-se de que ele seja uma constante ou retorne um erro (código 20)
    - Então, visite-o (`size`)
    - Em seguida, verifique se o tipo do tamanho do arranjo é inteiro ou retorne um erro (código 2)
    - Se o tamanho do arranjo não estiver definido, ele será inferido após a visita da inicialização no objeto `Decl`
- Crie o tipo do arranjo (`ArrayType`) usando o tipo de seus elementos e seu tamanho e atribua-o ao nó atual (`attrs['uc_type']`)

5. Declaração de função (`visit_FuncDecl`)

- Comece visitando o tipo (`type`)
- Em seguida, visite os argumentos (`args`), caso estejam definidos
    - Em seguida, copie a tabela de símbolos do escopo do protótipo da função para o nó atual (`attrs['symtab']`)
    - Depois disso, crie uma lista com os tipos de todos os argumentos
- Crie o tipo da função (`FunctionType`) usando seu tipo de retorno e a lista com os tipos de seus argumentos e atribua-o ao nó atual (`attrs['uc_type']`)

6. Lista de declarações (`visit_DeclList`)

- Crie uma tabela de símbolos para o escopo do bloco, vincule-a a tabela de símbolos anterior e atribua-a ao nó atual (`attrs['symtab']`)
- Visite as declarações (`decls`)

7. Tipo (`visit_Type`)

- Obtenha o objeto `uCType` básico correspondente ao tipo (`name`) e atribua-o ao nó atual (`attrs['uc_type']`)

### Comandos

1. If (`visit_If`)

- Primeiro, visite a condição (`cond`)
- Em seguida, verifique se a expressão condicional é do tipo booleano ou retorne um erro de tipo (código 19)
- Por fim, visite as declarações relacionadas ao "então" (`ifthen`) e ao "senão" (`ifelse`), se houver

```cpp
if(3) { } // SemanticError: The Condition expression must be of type 'bool'.
```

2. For (`visit_For`)

- Anexe o nó do laço atual ao atributo de lista usado para vincular o nó à instrução `break` (`attrs['loops']`)
- Visite a inicialização (`init`), se ela estiver definida
- Verifique se a condição (`cond`) está definida
    - Se a condição estiver definida, visite-a
    - Então, verifique se a expressão condicional é do tipo booleano ou retorne um erro de tipo (código 19)
- Visite o incremento (`next`), se ele estiver definido
- Visite o corpo do laço (`stmt`)
- Se a inicialização estiver definida e ela for uma lista (`DeclList`), restaure a referência à tabela de símbolos anterior
- Retire o nó do laço atual do atributo de lista usado para vincular o nó à instrução `break` (`attrs['loops']`)

3. While (`visit_While`)

- Anexe o nó do loop atual ao atributo de lista usado para vincular o nó à instrução `break` (`attrs['loops']`)
- Visite a condição (`cond`)
- Verifique se a expressão condicional é do tipo booleano ou retorne um erro de tipo (código 19)
- Visite o corpo do laço (`stmt`)
- Retire o nó do laço atual do atributo de lista usado para vincular o nó à instrução `break` (`attrs['loops']`)

4. Bloco (`visit_Compound`)

- Crie uma tabela de símbolos para o escopo do bloco, vincule-a a tabela de símbolos anterior e atribua-a ao nó atual (`attrs['symtab']`)
- Se estiver dentro de uma função e o escopo do bloco atual for o escopo do bloco do corpo da função, obtenha a tabela de símbolos associada ao escopo do protótipo da função criado durante a definição da função
    - Usando a tabela de símbolos associada ao escopo do protótipo da função, itere sobre a lista de parâmetros definidos na declaração da função
        - A cada iteração, insira o parâmetro na tabela de símbolos do escopo do bloco atual e atribua-o ao escopo do bloco atual
- Visite as declarações (`decls`)
- Visite os comandos (`stmts`) 
- Restaure a referência à tabela de símbolos anterior

5. Atribuição (`visit_Assignment`)

- Visite o lado direito (`rvalue`)
- Visite o lado esquerdo (`lvalue`)
- Verifique se o lado esquerdo é um identificador ou uma referência de matriz, caso contrário, retorne um erro (código 32)
- Verifique se a atribuição é permitida, caso contrário, retorne um erro de tipo (código 4): os lados esquerdo e direito de uma operação de atribuição devem ser declarados como do mesmo tipo (`attrs['uc_type']`)
- Verifique se o operador de atribuição (`op`) é suportado pelo tipo ou retorne um erro (código 5), tentativas de usar operadores não suportados devem resultar em erro
- Atribua o tipo do lado esquerdo ao nó atual (`attrs['uc_type']`)

Aqui está a lista completa de operadores suportados por cada tipo:

```python
    # int:
        assign_ops = {"="}

    # char:
        assign_ops = {"="}

    # bool:
        assign_ops = {"="}

    # array:
        assign_ops = {"="}
```

Veja o exemplo abaixo:

```cpp
char c = '3';
int j = c;             // SemanticError: Cannot assign type 'char' to type 'int'
```

6. Break (`visit_Break`)

- Primeiro, verifique se a instrução `break` está dentro de um laço, caso contrário, deve retornar um erro (código 8)
- Em seguida, vincule-o ao nó do laço atual (`attrs['loop']`)

7. Chamada de função (`visit_FuncCall`)

- Procure o nome (`name`) na tabela de símbolos do escopo do bloco atual
- Certifique-se de que ele está definido, caso contrário, retorne um erro (código 1)
- Verifique se o nome fornecido faz referência a uma função ou retorne um erro (código 16)
- Defina o tipo do nó atual para o mesmo tipo que o tipo de retorno da definição da função (`attrs['uc_type']`)
- Verifique se o argumento está definido (`args`)
    - Se o argumento estiver definido, visite-o (`args`)
    - Se o argumento for uma única expressão (não `ExprList`), crie uma lista com tal expressão, caso contrário, obtenha a lista de expressões
    - Se o argumento não estiver definido, inicialize a lista de expressões como vazia
- Obtenha a lista de valores da tabela de símbolos associada ao escopo do protótipo de função criado durante a definição da função (`attrs['symtab']`)
- Verifique se o número dos argumentos corresponde ao dos parâmetros na definição da função ou retorne um erro (código 17)
- Em seguida, itere sobre os elementos da lista de argumentos passados ​​para a chamada da função e a lista de parâmetros definidos na declaração da função
    - A cada iteração, verifique se o tipo do argumento corresponde ao do parâmetro ou retorne um erro de tipo (código 18)

8. Assert (`visit_Assert`)

- Visite a expressão (`expr`)
- Verifique se ela é do tipo booleano ou retorne um erro de tipo (código 3)

9. Comando vazio (`visit_EmptyStatement`)

- Não faça nada, apenas passe (`pass`).

10. Print (`visit_Print`)

- Visite a expressão (`expr`), se ela estiver definida
    - Se for uma expressão única (não `ExprList`), crie uma lista com tal expressão, caso contrário, obtenha a lista de expressões
    - Itere sobre as expressões da lista
        - A cada iteração, certifique-se de que a expressão seja de um tipo básico ou retorne um erro de tipo (código 21)

11. Read (`visit_Read`)

- Visite a expressão (`expr`), se ela estiver definida
- Se for uma expressão única (não `ExprList`), crie uma lista com tal expressão, caso contrário, obtenha a lista de expressões
- Itere sobre as expressões da lista
    - A cada iteração, verifique se a expressão é um identificador ou uma referência à arranjo, caso contrário, retorne um erro (código 31)
    - Em seguida, verifique se o identificador faz referência a uma variável ou retorne um erro (código 23)
    - Por fim, verifique se a variável é de tipo básico ou retorne um erro (código 22)

12. Return (`visit_Return`)

- Verifique se a expressão está definida (`expr`)
    - Se a expressão estiver definida, visite-a (`expr`)
    - Então, pegue o tipo da expressão se ela for única (não `ExprList`), caso contrário, pegue o tipo da última expressão da lista
    - Se a expressão não estiver definida, o retorno esperado será do tipo `void` (`VoidType`)
- Verifique se o tipo esperado para o retorno é idêntico ao tipo de retorno da definição da função ou retorne um erro de tipo (código 24)
- Defina o tipo do nó atual para o mesmo tipo que o tipo de retorno da definição da função (`attrs['uc_type']`)

### Expressões

1. Constante (`visit_Constant`)

- Obtenha o objeto `uCType` correspondente ao seu tipo (`type`) e atribua-o ao nó atual (`attrs['uc_type']`)

2. Identificador (`visit_ID`)

- Procure a declaração do identificador (`name`) na tabela de símbolos do escopo do bloco atual
- Certifique-se de que ele está definido, caso contrário retorne um erro (código 1)
- Por fim, inicialize os atributos do nó atual (`attrs`) usando informações da tabela de símbolos

3. Operação binária (`visit_BinaryOp`)

- Comece visitando cada operando da operação (`left`, `right`)
- Verifique se ambos os operandos possuem o mesmo tipo ou retorne um erro de tipo (código 6)
- Verifique se o operador (`op`) da operação binária é compatível com o tipo dos operandos, tentativas de usar operadores não suportados devem resultar em erro (código 7)
- Verifique o tipo do resultado:
    - Operações binárias usando operador aritmético produzem um resultado do mesmo tipo que os operandos
    - Operações binárias usando operador relacional produzem um resultado do tipo booleano (`BoolType`)
- Atribua o tipo do resultado ao nó atual (`attrs['uc_type']`)

Aqui está a lista completa de operadores suportados por cada tipo:

```python
    #int:
        binary_ops = {"+", "-", "*", "/", "%"}
        rel_ops = {"==", "!=", "<", ">", "<=", ">="}

    # char:
        rel_ops = {"==", "!=", "&&", "||"}

    # bool:
        rel_ops = {"==", "!=", "&&", "||"}

    # array:
        rel_ops = {"==", "!="}

    # string:
        binary_ops = {"+"}
        rel_ops = {"==", "!="}
```

Por exemplo:

```cpp
        int a = 2;
        char b = '3';

        int c = a + 3;    // OK
        int d = a + b;    // Error.  int + char
        int e = b + 4;    // Error.  char + int
        char a[] = "Hello" + "World";     // OK (concatenation) 
        char b[] = "Hello" * "World";     // Error: unsupported op *
```

4. Operação unária (`visit_UnaryOp`)

- Comece visitando o operando da operação (`expr`)
- Verifique se o operador (`op`) da operação é compatível com o tipo do operando, tentativas de usar operadores não suportados devem resultar em erro (código 26)
- Defina o tipo do nó atual que representa a operação unária com o mesmo tipo do operando (`attrs['uc_type']`)

Aqui está a lista completa de operadores suportados por cada tipo:

```python
    #int:
        unary_ops = {"-", "+"}

    # bool:
        unary_ops = {"!"}
```

5. Lista de Expressões (`visit_ExprList`)

- Basta visitar cada expressão (`exprs`)

6. Referência à arranjo (`visit_ArrayRef`)

- Comece visitando o subscrito (`subscript`)
- Verifique se o tipo do subscrito é inteiro (`IntType`) ou retorne um erro (código 2)
- Visite o nome (`name`)
- Verifique se o nome é do tipo arranjo (`ArrayType`) ou retorne um erro (código 30)
- Defina o tipo do nó atual para o mesmo tipo dos elementos do arranjo (`attrs['uc_type']`)

Veja o exemplo abaixo:

```cpp
int v[2] = {1, 2};
char f = '3';
int j = v[f];             // SemanticError: subscript must be of type 'int', not type 'char'
```

7. Lista de inicialização (`visit_InitList`)

- Crie uma lista vazia para armazenar os tipos dos elementos da lista de inicialização
- Crie uma lista vazia para armazenar os tamanhos dos elementos da lista de inicialização
- Itere sobre os elementos da lista de inicialização (`exprs`)
    - Em cada iteração, visite um elemento
    - Em seguida, verifique se ele também é uma lista (`InitList`) ou um escalar (não `InitList`)
        - Se for um escalar (não `InitList`), verifique se ele é uma constante (`Constant`) ou retorne um erro (código 20)
        - Em seguida, adicione seu tipo e tamanho às listas de tipos e tamanhos criadas no início:
            - Strings literais (`StringType`) são do tipo arranjo cujos elementos são do tipo `char` e o seu tamanho é o comprimento da string (excluindo as aspas)
            - Outros tipos são os básicos e seu tamanho é 1
        - Se for uma lista (`InitList`), adicione seu tipo e tamanho às listas de tipos e tamanhos criadas no início
            - Se ela for do tipo arranjo, seu tamanho é o tamanho do arranjo
            - Se ela for de um tipo básico, seu tamanho é 1
- Após a conclusão das iterações, avalie a lista de tamanhos criada no início e certifique-se de que todos os elementos tenham o mesmo tamanho ou retorne um erro (código 13)
- Além disso, avalie a lista de tipos criada no início e certifique-se de que todos os elementos tenham o mesmo tipo ou retorne um erro de tipo (código 29)
- Verifique se a lista de inicialização (`exprs`) é composta por uma única expressão do tipo string (`StringType`)
    - Se ela for composta por uma única expressão do tipo string, atribua esse tipo ao nó atual (`attrs['uc_type']`)
    - Se ela não for composta por uma única expressão do tipo string, use o seu tamanho e o tipo de seu primeiro elemento para criar um tipo arranjo e atribuí-lo ao nó atual (`attrs['uc_type']`)


## Esboço da Construção da AST

In [ ]:
!pip install sly
from sly import Lexer, Parser

Copie o código da classe `UCLexer` que você escreveu no [primeiro projeto](https://colab.research.google.com/drive/1TFZwdVJGMVDo48cq63LsWGnqC9UXx3n1?usp=sharing) e cole na célula abaixo.

In [ ]:
class UCLexer(Lexer):
    """A lexer for the uC language."""

    def __init__(self, error_func):
        """Create a new Lexer.
        An error function. Will be called with an error
        message, line and column as arguments, in case of
        an error during lexing.
        """
        self.error_func = error_func
        
    # Reserved keywords
    keywords = {
        'assert': "ASSERT",
        'break': "BREAK",
        'char': "CHAR",
        'else': "ELSE",
        'for': "FOR",
        'if': "IF",
        'int': "INT",
        'print': "PRINT",
        'read': "READ",
        'return': "RETURN",
        'void': "VOID",
        'while': "WHILE",
    }

    # All the tokens recognized by the lexer
    tokens = tuple(keywords.values()) + (
       # Identifiers
        "ID",
        # constants
        "INT_CONST",
        "CHAR_CONST",
        "STRING_LITERAL",
        # Operators
        "PLUS",
        "MINUS",
        "TIMES",
        "DIVIDE",
        "MOD",
        "LE",
        "LT",
        "GE",
        "GT",
        "EQ",
        "NE",
        "OR",
        "AND",
        "NOT",
        # Assignment
        "EQUALS",
        # Delimeters
        "LPAREN",
        "RPAREN",  # ( )
        "LBRACKET",
        "RBRACKET",  # [ ]
        "LBRACE",
        "RBRACE",  # { }
        "COMMA",
        "SEMI",  # , ;
    )

    # String containing ignored characters (between tokens)
    ignore = " \t"

    # Other ignored patterns
    ignore_newline = r'[\\\n]'# <<< INCLUDE A REGEX HERE FOR NEWLINE >>>
    ignore_comment = r'(\/\*([\s\S]*?)\*\/)|(\/\/.*\n)'# <<< INCLUDE A REGEX HERE FOR COMMENT >>>

    # Regular expression rules for tokens
    ID = r'[a-zA-Z_][0-9a-zA-Z_]*'# <<< INCLUDE A REGEX HERE FOR ID >>>
    INT_CONST = r'[0-9]+'# <<< INCLUDE A REGEX HERE FOR INT_CONST >>>
    CHAR_CONST = r'(\'(\\.|[^\\]?)\')' # <<< INCLUDE A REGEX HERE FOR CHAR_CONST >>>
    # <<< YOUR CODE HERE >>>
    STRING_LITERAL = r'(\"(.|(\\\"))*?[^\\]\")'
    PLUS = r'\+'
    MINUS = r'\-'
    TIMES = r'\*'
    DIVIDE = r'\/(?!\*)'
    MOD = r'\%'
    LE = r'\<\='
    LT = r'\<(?!\=)'
    GE = r'\>\='
    GT = r'\>(?!\=)'
    EQ = r'\=\='
    NE = r'\!\='
    OR = r'\|\|'
    AND = r'\&\&'
    NOT = r'\!(?!\=)'
    EQUALS = r'\=(?!\=)'
        # Delimeters
    LPAREN = r'\('
    RPAREN = r'\)'
    LBRACKET = r'\['
    RBRACKET = r'\]'
    LBRACE = r'\{'
    RBRACE = r'\}'
    COMMA = r'\,'
    SEMI = r'\;'

    #erros
    unmatched_quote = r'\"[^\"]*?(?=\n)'

    def unmatched_quote(self, t):
      msg = "Unterminated string literal"
      self._error(msg, t)

    uccomment = r'(\/\*)([\s\S]*?)[^(\*\/)]\Z'

    def uccomment(self, t):
      msg = "Unterminated comment"
      self._error(msg, t)

    Unterminated_character = r'\'[^\']*?(?=\n)|\'.{2,}\'|\'\\\''

    def Unterminated_character(self, t):
      msg = "Unterminated character const"
      self._error(msg, t)

    # Special cases
    def ID(self, t):
      t.type = self.keywords.get(t.value, "ID")
      return t

    # Define a rule so we can track line numbers
    def ignore_newline(self, t):
        self.lineno += len(t.value)
 
    def ignore_comment(self, t):
        self.lineno += t.value.count("\n")

    def find_column(self, token):
        """Find the column of the token in its line."""
        last_cr = self.text.rfind('\n', 0, token.index)
        return token.index - last_cr

    # Internal auxiliary methods
    def _error(self, msg, token):
        location = self._make_location(token)
        self.error_func(msg, location[0], location[1])
        self.index += 1

    def _make_location(self, token):
        return token.lineno, self.find_column(token)

    # Error handling rule
    def error(self, t):
        msg = "Illegal character %s" % repr(t.value[0])
        self._error(msg, t)

    # Scanner (used only for test)
    def scan(self, text):
        output = ""
        for tok in self.tokenize(text):
            print(tok)
            output += str(tok) + "\n"
        return output

Copie o código das classes de nós AST que você escreveu no [terceiro projeto](https://colab.research.google.com/drive/1rg6TtYoPCAkzvf7QSSF1Tiof997uVO2a?usp=sharing) e cole na célula abaixo.

In [ ]:
import sys


def represent_node(obj, indent):
    def _repr(obj, indent, printed_set):
        """
        Get the representation of an object, with dedicated pprint-like format for lists.
        """
        if isinstance(obj, list):
            indent += 1
            sep = ",\n" + (" " * indent)
            final_sep = ",\n" + (" " * (indent - 1))
            return (
                "["
                + (sep.join((_repr(e, indent, printed_set) for e in obj)))
                + final_sep
                + "]"
            )
        elif isinstance(obj, Node):
            if obj in printed_set:
                return ""
            else:
                printed_set.add(obj)
            result = obj.__class__.__name__ + "("
            indent += len(obj.__class__.__name__) + 1
            attrs = []
            for name in obj.__slots__:
                if name == "bind":
                    continue
                value = getattr(obj, name)
                value_str = _repr(value, indent + len(name) + 1, printed_set)
                attrs.append(name + "=" + value_str)
            sep = ",\n" + (" " * indent)
            final_sep = ",\n" + (" " * (indent - 1))
            result += sep.join(attrs)
            result += ")"
            return result
        elif isinstance(obj, str):
            return obj
        else:
            return ""

    # avoid infinite recursion with printed_set
    printed_set = set()
    return _repr(obj, indent, printed_set)

class Node:
    """Abstract base class for AST nodes."""

    __slots__ = ("coord", "attrs",)

    def __init__(self, coord=None):
        self.coord = coord
        self.attrs = {}

    def children(self):
        """A sequence of all children that are Nodes"""
        pass

    attr_names = ()

    def __repr__(self):
        """Generates a python representation of the current node"""
        return represent_node(self, 0)
  
    def show(
        self,
        buf=sys.stdout,
        offset=0,
        attrnames=False,
        nodenames=False,
        showcoord=False,
        _my_node_name=None,
    ):
        """Pretty print the Node and all its attributes and children (recursively) to a buffer.
        buf:
            Open IO buffer into which the Node is printed.
        offset:
            Initial offset (amount of leading spaces)
        attrnames:
            True if you want to see the attribute names in name=value pairs. False to only see the values.
        nodenames:
            True if you want to see the actual node names within their parents.
        showcoord:
            Do you want the coordinates of each Node to be displayed.
        """
        lead = " " * offset
        if nodenames and _my_node_name is not None:
            buf.write(lead + self.__class__.__name__ + " <" + _my_node_name + ">: ")
            inner_offset = len(self.__class__.__name__ + " <" + _my_node_name + ">: ")
        else:
            buf.write(lead + self.__class__.__name__ + ":")
            inner_offset = len(self.__class__.__name__ + ":")

        if self.attr_names:
            if attrnames:
                nvlist = [
                    (n, represent_node(getattr(self, n), offset+inner_offset+1+len(n)+1))
                    for n in self.attr_names
                    if getattr(self, n) is not None
                ]
                attrstr = ", ".join("%s=%s" % nv for nv in nvlist)
            else:
                vlist = [getattr(self, n) for n in self.attr_names]
                attrstr = ", ".join(
                    represent_node(v, offset + inner_offset + 1) for v in vlist
                )
            buf.write(" " + attrstr)

        if showcoord:
            if self.coord and self.coord.line != 0:
                buf.write(" %s" % self.coord)
        buf.write("\n")

        for (child_name, child) in self.children():
            child.show(buf, offset + 4, attrnames, nodenames, showcoord, child_name)

class ArrayDecl(Node):

    # <<< YOUR CODE HERE >>>
    __slots__ = ("type", "size",)

    def __init__(self, type, size, coord=None):
        super().__init__(coord)
        self.type = type
        self.size = size

    def children(self):
        nodelist = []
        if self.type is not None:
            nodelist.append(('type', self.type))
        if self.size is not None:
            nodelist.append(('size', self.size))
        return tuple(nodelist)



class ArrayRef(Node):

    # <<< YOUR CODE HERE >>>
    __slots__ = ("name", "subscript",)

    def __init__(self, name, subscript, coord=None):
        super().__init__(coord)
        self.name = name
        self.subscript = subscript

    def children(self):
        nodelist = []
        if self.name is not None:
            nodelist.append(('name', self.name))
        if self.subscript is not None:
            nodelist.append(('subscript', self.subscript))
        return tuple(nodelist)

class Assert(Node):

    # <<< YOUR CODE HERE >>>
    __slots__ = ("expr",)

    def __init__(self, expr, coord=None):
        super().__init__(coord)
        self.expr = expr

    def children(self):
        nodelist = []
        if self.expr is not None:
            nodelist.append(('expr', self.expr))
        return tuple(nodelist)

class Assignment(Node):

    # <<< YOUR CODE HERE >>>
    __slots__ = ("op","lvalue", "rvalue",)

    def __init__(self, op, lvalue, rvalue, coord=None):
        super().__init__(coord)
        self.op = op
        self.lvalue = lvalue
        self.rvalue = rvalue

    def children(self):
        nodelist = []
        if self.lvalue is not None:
            nodelist.append(('lvalue', self.lvalue))
        if self.rvalue is not None:
            nodelist.append(('rvalue', self.rvalue))
        return tuple(nodelist)

    attr_names = ("op",)

class BinaryOp(Node):

    __slots__ = ("op", "left", "right",)

    def __init__(self, op, left, right, coord=None):
        super().__init__(coord)
        self.op = op
        self.left = left
        self.right = right

    def children(self):
        nodelist = []
        if self.left is not None:
            nodelist.append(('left', self.left))
        if self.right is not None:
            nodelist.append(('right', self.right))
        return tuple(nodelist)

    attr_names = ("op",)

class Break(Node):

    # <<< YOUR CODE HERE >>>
    __slots__ = ()

    def __init__(self, coord=None):
        super().__init__(coord)
       

    def children(self):
        nodelist = []
        
        return tuple(nodelist)

    

class Cast(Node):   #eu add esse

    __slots__ = ("type", "expr",)

    def __init__(self, type, expr, coord=None):
        super().__init__(coord)
        self.type = type
        self.expr = expr

    def children(self):
        nodelist = []
        if self.type is not None:
            nodelist.append(('type', self.type))
        if self.expr is not None:
            nodelist.append(('expr', self.expr))
        return tuple(nodelist)
class Compound(Node):

    # <<< YOUR CODE HERE >>>
    __slots__ = ("dcls", "stmts")

    def __init__(self, dcls, stmts, coord=None):
        super().__init__(coord)
        self.dcls = dcls
        self.stmts = stmts

    def children(self):
        nodelist = []
        for i, child in enumerate(self.dcls or []):
            nodelist.append(('dcls[%d]' % i, child))
        for i, child in enumerate(self.stmts or []):
            nodelist.append(('stmts[%d]' % i, child))    
        return tuple(nodelist)



class Constant(Node):

    __slots__ = ("type", "value",)

    def __init__(self, type, value, coord=None):
        super().__init__(coord)
        self.type = type
        self.value = value

    def children(self):
        return ()

    attr_names = ("type", "value",)

class Decl(Node):

    # <<< YOUR CODE HERE >>>
    __slots__ = ("name", "type", "init",)

    def __init__(self, name, type, init, coord=None):
        super().__init__(coord)
        self.name = name
        self.type = type
        self.init = init

    def children(self):
        nodelist = []
        if self.type is not None:
            nodelist.append(('type', self.type))
        if self.init is not None:
            nodelist.append(('init', self.init))
        return tuple(nodelist)

    attr_names = ("name",)

class DeclList(Node):

    # <<< YOUR CODE HERE >>>
    __slots__ = ("decls", )

    def __init__(self, decls, coord=None):
        super().__init__(coord)
        self.decls = decls
        

    def children(self):
        nodelist = []
        for i, child in enumerate(self.decls or []):
            nodelist.append(('decls[%d]' % i, child))   
        return tuple(nodelist)
class EmptyStatement(Node):

    # <<< YOUR CODE HERE >>>
    __slots__ = ()

    def __init__(self, coord=None):
        super().__init__(coord)
       

    def children(self):
        nodelist = []
        
        return tuple(nodelist)

class ExprList(Node):

    # <<< YOUR CODE HERE >>>
    __slots__ = ("exprs", )

    def __init__(self, exprs, coord=None):
        super().__init__(coord)
        self.exprs = exprs
        

    def children(self):
        nodelist = []
        for i, child in enumerate(self.exprs or []):
            nodelist.append(('exprs[%d]' % i, child))   
        return tuple(nodelist)

class For(Node):

    # <<< YOUR CODE HERE >>>
    __slots__ = ("init", "cond", "next", "stmt",)

    def __init__(self, init, cond, next, stmt, coord=None):
        super().__init__(coord)
        self.init = init
        self.cond = cond
        self.next = next
        self.stmt = stmt

    def children(self):
        nodelist = []
        if self.init is not None:
            nodelist.append(('init', self.init))
        if self.cond is not None:
            nodelist.append(('cond', self.cond))
        if self.next is not None:
            nodelist.append(('next', self.next))    
        if self.stmt is not None:
            nodelist.append(('stmt', self.stmt))    
        return tuple(nodelist)

class FuncCall(Node):

    # <<< YOUR CODE HERE >>>
    __slots__ = ("name", "args",)

    def __init__(self, name, args, coord=None):
        super().__init__(coord)
        self.name = name
        self.args = args

    def children(self):
        nodelist = []
        if self.name is not None:
            nodelist.append(('name', self.name))
        if self.args is not None:
            nodelist.append(('args', self.args))
        return tuple(nodelist)

class FuncDecl(Node):

    # <<< YOUR CODE HERE >>>
    __slots__ = ("args", "type",)

    def __init__(self, args, type, coord=None):
        super().__init__(coord)
        self.args = args
        self.type = type

    def children(self):
        nodelist = []
        if self.args is not None:
            nodelist.append(('args', self.args))
        if self.type is not None:
            nodelist.append(('type', self.type))
        return tuple(nodelist)

class FuncDef(Node):

    # <<< YOUR CODE HERE >>>
    __slots__ = ("spec", "decl", "body")

    def __init__(self, spec, decl, body, coord=None):
        super().__init__(coord)
        self.spec = spec
        self.decl = decl
        self.body = body

    def children(self):
        nodelist = []
        if self.spec is not None:
            nodelist.append(('spec', self.spec))
        if self.decl is not None:
            nodelist.append(('decl', self.decl))
       # for i, child in enumerate(self.param_decls or []):
       #     nodelist.append(('param_decls[%d]' % i, child))   
        if self.body is not None:
            nodelist.append(('body', self.body))      
        return tuple(nodelist)

class GlobalDecl(Node):

    # <<< YOUR CODE HERE >>>
    __slots__ = ("decls", )

    def __init__(self, decls, coord=None):
        super().__init__(coord)
        self.decls = decls
        

    def children(self):
        nodelist = []
        for i, child in enumerate(self.decls or []):
            nodelist.append(('decls[%d]' % i, child))   
        return tuple(nodelist)

class ID(Node):

    # <<< YOUR CODE HERE >>>
    __slots__ = ("name",)

    def __init__(self, name, coord=None):
        super().__init__(coord)
        self.name = name
        
    def children(self):
        nodelist = []
 
        return tuple(nodelist)

    attr_names = ("name",)


class If(Node):

    # <<< YOUR CODE HERE >>>
    __slots__ = ("cond", "ifthen", "ifelse",)

    def __init__(self, cond, ifthen, ifelse, coord=None):
        super().__init__(coord)
        self.cond = cond
        self.ifthen = ifthen
        self.ifelse = ifelse

    def children(self):
        nodelist = []
        if self.cond is not None:
            nodelist.append(('cond', self.cond))
        if self.ifthen is not None:
            nodelist.append(('ifthen', self.ifthen))
        if self.ifelse is not None:
            nodelist.append(('ifelse', self.ifelse))
        return tuple(nodelist)

class InitList(Node):

    # <<< YOUR CODE HERE >>>
    __slots__ = ("exprs", )

    def __init__(self, exprs, coord=None):
        super().__init__(coord)
        self.exprs = exprs
        

    def children(self):
        nodelist = []
        for i, child in enumerate(self.exprs or []):
            nodelist.append(('exprs[%d]' % i, child))   
        return tuple(nodelist)

class ParamList(Node):

    # <<< YOUR CODE HERE >>>
    __slots__ = ("params", )

    def __init__(self, params, coord=None):
        super().__init__(coord)
        self.params = params
        

    def children(self):
        nodelist = []
        for i, child in enumerate(self.params or []):
            nodelist.append(('params[%d]' % i, child))   
        return tuple(nodelist)


class Print(Node):

    # <<< YOUR CODE HERE >>>
    __slots__ = ("expr",)

    def __init__(self, expr, coord=None):
        super().__init__(coord)
        self.expr = expr

    def children(self):
        nodelist = []
        if self.expr is not None:
            nodelist.append(('expr', self.expr))
        return tuple(nodelist)

class Program(Node):

    __slots__ = ("gdecls",)

    def __init__(self, gdecls, coord=None):
        super().__init__(coord)
        self.gdecls = gdecls

    def children(self):
        nodelist = []
        for i, child in enumerate(self.gdecls or []):
            nodelist.append(('gdecls[%d]' % i, child))
        return tuple(nodelist)

class Read(Node):

    # <<< YOUR CODE HERE >>>
    __slots__ = ("expr",)

    def __init__(self, expr, coord=None):
        super().__init__(coord)
        self.expr = expr

    def children(self):
        nodelist = []
        if self.expr is not None:
            nodelist.append(('expr', self.expr))
        return tuple(nodelist)

class Return(Node):

    # <<< YOUR CODE HERE >>>
    __slots__ = ("expr",)

    def __init__(self, expr, coord=None):
        super().__init__(coord)
        self.expr = expr

    def children(self):
        nodelist = []
        if self.expr is not None:
            nodelist.append(('expr', self.expr))
        return tuple(nodelist)

class Type(Node):

    # <<< YOUR CODE HERE >>>
    __slots__ = ("name",)

    def __init__(self, name, coord=None):
        super().__init__(coord)
        self.name = name
        
    def children(self):
        nodelist = []
 
        return tuple(nodelist)

    attr_names = ("name",)


class VarDecl(Node):

    # <<< YOUR CODE HERE >>>
     # <<< YOUR CODE HERE >>>
    __slots__ = ("declname", "type",)

    def __init__(self, declname=None, type=None, coord=None):
        super().__init__(coord)
        self.declname = declname
        self.type = type

    def children(self):
        nodelist = []
        if self.type is not None:
            nodelist.append(('type', self.type))
        
        return tuple(nodelist)

   

class UnaryOp(Node):

    # <<< YOUR CODE HERE >>>
    __slots__ = ("op", "expr",)

    def __init__(self, op, expr, coord=None):
        super().__init__(coord)
        self.op = op
        self.expr = expr

    def children(self):
        nodelist = []
        if self.expr is not None:
            nodelist.append(('expr', self.expr))
        
        return tuple(nodelist)

    attr_names = ("op",)

class While(Node):

    # <<< YOUR CODE HERE >>>
    __slots__ = ("cond", "stmt",)

    def __init__(self, cond, stmt, coord=None):
        super().__init__(coord)
        self.cond = cond
        self.stmt = stmt

    def children(self):
        nodelist = []
        if self.cond is not None:
            nodelist.append(('cond', self.cond))
        if self.stmt is not None:
            nodelist.append(('stmt', self.stmt))
        return tuple(nodelist)

Copie o código da classe `UCParser` que você escreveu no [terceiro projeto](https://colab.research.google.com/drive/1rg6TtYoPCAkzvf7QSSF1Tiof997uVO2a?usp=sharing) e cole na célula abaixo.

In [ ]:
class Coord:
    """Coordinates of a syntactic element. Consists of:
    - Line number
    - (optional) column number, for the Lexer
    """

    __slots__ = ("line", "column")

    def __init__(self, line, column=None):
        self.line = line
        self.column = column

    def __str__(self):
        if self.line and self.column is not None:
            coord_str = "@ %s:%s" % (self.line, self.column)
        elif self.line:
            coord_str = "@ %s" % (self.line)
        else:
            coord_str = ""
        return coord_str
        #eu add esse coord
class UCParser(Parser):
    """A parser for the uC language."""

    # Get the token list from the lexer (required)
    tokens = UCLexer.tokens

    precedence = (
        # <<< YOUR CODE HERE >>>
        ('left', 'ELSE'), 
        ('left', 'FOR'),
        ('left', 'WHILE'),  
        ('left', 'BREAK'),
        ('left', 'READ', 'PRINT'),
        ('left', 'RPAREN', 'LPAREN', 'LBRACKET', 'RBRACKET'),
        ('left', 'LBRACE'),
        ('left', 'RBRACE'),
        ('right', 'NOT'),
        ('left', 'AND', 'OR'),
        ('left', 'EQ', 'NE'),
        ('nonassoc', 'LE', 'LT', 'GE', 'GT'),
        ('left', 'PLUS', 'MINUS'),
        ('left', 'DIVIDE','TIMES', 'MOD'),
        ('right', 'EQUALS')
    )
    
    def __init__(self, error_func=lambda msg, x, y: print("Lexical error: %s at %d:%d" % (msg, x, y), file=sys.stdout)):
        """Create a new Parser.
        An error function for the lexer.
        """
        self.lexer = UCLexer(error_func)

    def parse(self, text, lineno=1, index=0):
        return super().parse(self.lexer.tokenize(text, lineno, index))

    # Internal auxiliary methods
    def _token_coord(self, p):
        line, column = self.lexer._make_location(p)
        return Coord(line, column)

    def _build_declarations(self, spec, decls):
        """ Builds a list of declarations all sharing the given specifiers.
        """
        declarations = []

        for decl in decls:
            assert decl['decl'] is not None
            declaration = Decl(
                    name=None,
                    type=decl['decl'],
                    init=decl.get('init'))

            fixed_decl = self._fix_decl_name_type(declaration, spec)
            declarations.append(fixed_decl)

        return declarations

    def _fix_decl_name_type(self, decl, typename):
        """ Fixes a declaration. Modifies decl.
        """
        # Reach the underlying basic type
        type = decl
        while not isinstance(type, VarDecl):
            type = type.type

        decl.name = type.declname
        if not typename:
            # Functions default to returning int
            if not isinstance(decl.type, FuncDecl):
                self._parse_error("Missing type in declaration", decl.coord)
            type.type = Type('int', coord=decl.coord)
        else:
            type.type = Type(typename.name, coord=typename.coord)
            
        return decl
        
    def _type_modify_decl(self, decl, modifier):
        """Tacks a type modifier on a declarator, and returns
           the modified declarator.
           Note: the declarator and modifier may be modified
        """
        modifier_head = modifier
        modifier_tail = modifier

        # The modifier may be a nested list. Reach its tail.
        while modifier_tail.type:
            modifier_tail = modifier_tail.type

        # If the decl is a basic type, just tack the modifier onto it
        if isinstance(decl, VarDecl):
            modifier_tail.type = decl
            return modifier
        else:
            # Otherwise, the decl is a list of modifiers. Reach
            # its tail and splice the modifier onto the tail,
            # pointing to the underlying basic type.
            decl_tail = decl

            while not isinstance(decl_tail.type, VarDecl):
                decl_tail = decl_tail.type

            modifier_tail.type = decl_tail.type
            decl_tail.type = modifier_head
            return decl

    # Error handling rule
    def error(self, p):
        if p:
            if hasattr(p, 'lineno'):
                print("Error at line %d near the symbol %s " % (p.lineno, p.value))
            else:
                print("Error near the symbol %s" % p.value)
        else:
            print("Error at the end of input")

    # <program> ::= {<global_declaration>}+
    @_('global_declaration_list')
    def program(self, p):
        return Program(p.global_declaration_list)
        
    @_('global_declaration { global_declaration }')
    def global_declaration_list(self, p):
        return [p.global_declaration0] + p.global_declaration1        

    # <global_declaration> ::= <function_definition>
    #                        | <declaration>
    @_('function_definition',
       'declaration')
    def global_declaration(self, p):
        if hasattr(p, 'function_definition'):
            return p.function_definition
        else:
            return GlobalDecl(p.declaration)

 # <function_definition> ::= <type_specifier> <declarator> <compound_statement>
    # <<< YOUR CODE HERE >>>
    @_('type_specifier declarator compound_statement' )
    def function_definition(self, p):
        return (FuncDef(p.type_specifier, self._build_declarations(spec=p.type_specifier, decls=[dict(decl= p.declarator, init=None)])[0], p.compound_statement))  
    # <type_specifier> ::= "void"
    #                    | "char"
    #                    | "int"
    # <<< YOUR CODE HERE >>>
    @_('VOID')
    def type_specifier(self, p):
        return (Type('void', coord=self._token_coord(p)))

    @_('CHAR')
    def type_specifier(self, p):
       return (Type('char', coord=self._token_coord(p))) 

    @_('INT')
    def type_specifier(self, p):
        return (Type('int', coord=self._token_coord(p)))    

    # <declarator> ::= <identifier>
    #                | <declarator> "[" {<constant_expression>}? "]"
    #                | <declarator> "(" {<parameter_list>}? ")"
    # <<< YOUR CODE HERE >>>
    @_('ID')
    def declarator(self, p):
        return (VarDecl(ID(str(p.ID), coord= self._token_coord(p))))  #faltou testar. Não temos 100% de certeza aqui

    @_('declarator LBRACKET [ constant_expression ] RBRACKET')
    def declarator(self, p):
        return (self._type_modify_decl(decl=p.declarator, modifier=ArrayDecl(None, p.constant_expression, coord=self._token_coord(p))))    

    @_('declarator LPAREN [ parameter_list ] RPAREN')
    def declarator(self, p):
        return (self._type_modify_decl(decl=p.declarator, modifier=FuncDecl(p.parameter_list, None, coord=self._token_coord(p))))  

   
    # <constant_expression> ::= <binary_expression>
    # <<< YOUR CODE HERE >>>
    @_('binary_expression')
    def constant_expression(self, p):
        return (p.binary_expression)

    # <binary_expression> ::= <unary_expression>
    #                       | <binary_expression>  "*"   <binary_expression>
    #                       | <binary_expression>  "/"   <binary_expression>
    #                       | <binary_expression>  "%"   <binary_expression>
    #                       | <binary_expression>  "+"   <binary_expression>
    #                       | <binary_expression>  "-"   <binary_expression>
    #                       | <binary_expression>  "<"   <binary_expression>
    #                       | <binary_expression>  "<="  <binary_expression>
    #                       | <binary_expression>  ">"   <binary_expression>
    #                       | <binary_expression>  ">="  <binary_expression>
    #                       | <binary_expression>  "=="  <binary_expression>
    #                       | <binary_expression>  "!="  <binary_expression>
    #                       | <binary_expression>  "&&"  <binary_expression>
    #                       | <binary_expression>  "||"  <binary_expression>
    # <<< YOUR CODE HERE >>>
    @_('unary_expression')
    def binary_expression(self, p):
        return (p.unary_expression)

    @_('binary_expression TIMES binary_expression', 
       'binary_expression DIVIDE binary_expression', 
       'binary_expression MOD binary_expression',
       'binary_expression PLUS binary_expression',
       'binary_expression MINUS binary_expression', 
       'binary_expression LT binary_expression',
       'binary_expression LE binary_expression',
       'binary_expression GT binary_expression',
       'binary_expression GE binary_expression',
       'binary_expression EQ binary_expression',
       'binary_expression NE binary_expression',
       'binary_expression AND binary_expression',
       'binary_expression OR binary_expression')
    def binary_expression(self, p):
        return (BinaryOp(p[1], p.binary_expression0, p.binary_expression1, coord=p.binary_expression0.coord))   #testar isso aqui tambem

    # <unary_expression> ::= <postfix_expression>
    #                      | <unary_operator> <unary_expression>
    # <<< YOUR CODE HERE >>>
    @_('postfix_expression')
    def unary_expression(self, p):
        return (p.postfix_expression)

    @_('unary_operator unary_expression')
    def unary_expression(self, p):
        return (UnaryOp(p.unary_operator, p.unary_expression, coord= p.unary_expression.coord))   

    # <postfix_expression> ::= <primary_expression>
    #                        | <postfix_expression> "[" <expression> "]"
    #                        | <postfix_expression> "(" {<argument_expression>}? ")"
    # <<< YOUR CODE HERE >>>
    @_('primary_expression')
    def postfix_expression(self, p):
        return (p.primary_expression)

    @_('postfix_expression LBRACKET expression RBRACKET')
    def postfix_expression(self, p):
        return (ArrayRef(p.postfix_expression, p.expression, coord= p.postfix_expression.coord))   

    @_('postfix_expression LPAREN [ argument_expression ] RPAREN ')
    def postfix_expression(self, p):
        return (FuncCall(p.postfix_expression, p.argument_expression, coord=p.postfix_expression.coord))  

    # <primary_expression> ::= <identifier>
    #                        | <constant>
    #                        | <string>
    #                        | "(" <expression> ")"
    # <<< YOUR CODE HERE >>>
    @_('ID')
    def primary_expression(self, p):
        return (ID(str(p.ID), coord=self._token_coord(p))) #conferir depois
    
    @_('constant')
    def primary_expression(self, p):
        return (p.constant)   
    
    @_('STRING_LITERAL')
    def primary_expression(self, p):
        return (Constant('string', str(p.STRING_LITERAL), coord=self._token_coord(p)))  

    @_('LPAREN expression RPAREN')
    def primary_expression(self, p):
        return (p.expression)

    # <constant> ::= <integer_constant>
    #              | <character_constant>
    # <<< YOUR CODE HERE >>>
    @_('INT_CONST')
    def constant(self, p):
        return (Constant('int', str(p.INT_CONST), coord=self._token_coord(p)))
   
    @_('CHAR_CONST')
    def constant(self, p):
        return (Constant('char', str(p.CHAR_CONST), coord=self._token_coord(p)))    

    # <expression> ::= <assignment_expression>
    #                | <expression> "," <assignment_expression>
    # <<< YOUR CODE HERE >>>
    @_('assignment_expression')
    def expression(self, p):
        return ( p.assignment_expression)
    @_('assignment_expression COMMA assignment_expression { COMMA assignment_expression }')
    def expression(self, p):
        return ExprList([p.assignment_expression0] + [p.assignment_expression1] + p.assignment_expression2, coord=p.assignment_expression0.coord)
    # <argument_expression> ::= <assignment_expression>
    #                         | <argument_expression> "," <assignment_expression>
    # <<< YOUR CODE HERE >>>
    @_('assignment_expression')
    def argument_expression(self, p):
        return ( p.assignment_expression)
    @_('assignment_expression COMMA assignment_expression { COMMA assignment_expression }')
    def argument_expression(self, p):
        return ExprList([p.assignment_expression0] + [p.assignment_expression1] + p.assignment_expression2, coord=p.assignment_expression0.coord)

    # <assignment_expression> ::= <binary_expression>
    #                           | <unary_expression> "=" <assignment_expression>
    # <<< YOUR CODE HERE >>>
    @_('binary_expression')
    def assignment_expression(self, p):
        return (p.binary_expression)

    @_('unary_expression EQUALS assignment_expression')
    def assignment_expression(self, p):
        return (Assignment('=', p.unary_expression, p.assignment_expression, coord=p.unary_expression.coord))

    # <unary_operator> ::= "+"
    #                    | "-"
    #                    | "!"
    # <<< YOUR CODE HERE >>>
    @_('PLUS')
    def unary_operator(self, p):
        return ('+') 

    @_('MINUS')
    def unary_operator(self, p):
        return ('-')     

    @_('NOT')
    def unary_operator(self, p):
        return ('!') 
    # <parameter_list> ::= <parameter_declaration>
    #                    | <parameter_list> "," <parameter_declaration>
    # <<< YOUR CODE HERE >>>
   # @_('')   #checar
   # def parameter_list(self, p):
   #     return (p.parameter_list)

    @_('parameter_declaration { COMMA parameter_declaration }')
    def parameter_list(self, p):
        return (ParamList([p.parameter_declaration0] + p.parameter_declaration1 , coord=p.parameter_declaration0.coord)
)   

    # <parameter_declaration> ::= <type_specifier> <declarator>
    # <<< YOUR CODE HERE >>>
    @_('type_specifier declarator')
    def parameter_declaration(self, p):
        return (self._build_declarations(spec=p.type_specifier, decls=[dict(decl=p.declarator, init=None)])[0])

    # <declaration> ::=  <type_specifier> {<init_declarator_list>}? ";"
    # <<< YOUR CODE HERE >>>
    @_('type_specifier [ init_declarator_list ] SEMI')
    def declaration(self, p):
        return (self._build_declarations(spec=p.type_specifier, decls=p.init_declarator_list))

    # <init_declarator_list> ::= <init_declarator>
    #                          | <init_declarator_list> "," <init_declarator>
    # <<< YOUR CODE HERE >>>
    #@_('init_declarator_list')                 #eu comentei pra testar
    #def init_declarator_list(self, p):
    #    return (list(p.init_declarator))

    @_('init_declarator { COMMA init_declarator }')
    def init_declarator_list(self, p):
        return ([ p.init_declarator0 ]+ p.init_declarator1)  

    # <init_declarator> ::= <declarator>
    #                     | <declarator> "=" <initializer>
    # <<< YOUR CODE HERE >>>
   
    @_('declarator [ EQUALS initializer ]')
    def init_declarator(self, p):
        return (dict(decl=p.declarator, init=p.initializer))

    # <initializer> ::= <assignment_expression>
    #                 | "{" {<initializer_list>}? "}"
    #                 | "{" <initializer_list> , "}"
    # <<< YOUR CODE HERE >>>
    @_('assignment_expression')
    def initializer(self, p):
        return (p.assignment_expression)
    @_('LBRACE [ initializer_list ] RBRACE')
    def initializer(self, p):
        return (p.initializer_list)   
    @_('LBRACE initializer_list "," RBRACE')
    def initializer(self, p):
        return (p.initializer_list)     

    # <initializer_list> ::= <initializer>
    #                      | <initializer_list> "," <initializer>
    # <<< YOUR CODE HERE >>>
   # @_('initializer_list')
    #def initializer_list(self, p):
     #   return (p.initializer_list) 

    @_('initializer {  COMMA initializer }')        #eu comentei
    def initializer_list(self, p):
        return InitList([ p.initializer0 ] + p.initializer1, coord = p.initializer0.coord)    
#InitList(list(initializer), coord=initializer.coord
    # <compound_statement> ::= "{" {<declaration>}* {<statement>}* "}"
    # <<< YOUR CODE HERE >>>
    @_('LBRACE { declaration } { statement } RBRACE')
    def compound_statement(self, p):
        decl = []
        for sublist in p.declaration:
          for declarationflat in sublist:
            decl.append(declarationflat)
        #return Compound((declarationflat for sublist in p.declaration for declarationflat in sublist), p.statement)
        return Compound(decl, p.statement)
    # <statement> ::= <expression_statement>
    #               | <compound_statement>
    #               | <selection_statement>
    #               | <iteration_statement>
    #               | <jump_statement>
    #               | <assert_statement>
    #               | <print_statement>
    #               | <read_statement>
    # <<< YOUR CODE HERE >>>
    @_('expression_statement', 'compound_statement', 'selection_statement', 'iteration_statement', 'jump_statement', 'assert_statement', 'print_statement', 'read_statement')
    def statement(self, p):
        return (p[0]) 
    # <expression_statement> ::= {<expression>}? ";"
    # <<< YOUR CODE HERE >>>
    @_('[ expression ] SEMI')
    def expression_statement(self, p):
        return (p.expression if p.expression is not None else EmptyStatement(coord=self._token_coord(p))) 

    # <selection_statement> ::= "if" "(" <expression> ")" <statement>
    #                         | "if" "(" <expression> ")" <statement> "else" <statement>
    # <<< YOUR CODE HERE >>>
    @_('IF LPAREN expression RPAREN statement [ ELSE statement ]')
    def selection_statement(self, p):
        return (If(p.expression, p.statement0, p.statement1, coord=self._token_coord(p))) 

    # <iteration_statement> ::= "while" "(" <expression> ")" <statement>
    #                         | "for" "(" {<expression>}? ";" {<expression>}? ";" {<expression>}? ")" <statement>
    #                         | "for" "(" <declaration> {<expression>}? ";" {<expression>}? ")" <statement>
    # <<< YOUR CODE HERE >>>
    @_('WHILE LPAREN expression RPAREN statement')
    def iteration_statement(self, p):
        return (While(p.expression, p.statement, coord=self._token_coord(p)))

    @_('FOR LPAREN [ expression ] SEMI [ expression ] SEMI [ expression ] RPAREN statement')
    def iteration_statement(self, p):
        return (For(p.expression0, p.expression1, p.expression2, p.statement, coord=self._token_coord(p)))

    @_('FOR LPAREN declaration [ expression ] SEMI [ expression ] RPAREN statement')
    def iteration_statement(self, p):
        return (For(DeclList(p.declaration, coord=self._token_coord(p)), p.expression0, p.expression1, p.statement, coord=self._token_coord(p)))    

    # <jump_statement> ::= "break" ";"
    #                    | "return" {<expression>}? ";"
    # <<< YOUR CODE HERE >>>
    @_('BREAK SEMI')
    def jump_statement(self, p):
        return (Break(coord=self._token_coord(p)))

    @_('RETURN [ expression ] SEMI')
    def jump_statement(self, p):
        return (Return(p.expression, coord=self._token_coord(p)))

    # <assert_statement> ::= "assert" <expression> ";"
    # <<< YOUR CODE HERE >>>
    @_('ASSERT expression SEMI')
    def assert_statement(self, p):
        return (Assert(p.expression, coord=self._token_coord(p)))

    # <print_statement> ::= "print" "(" {<expression>}? ")" ";"
    # <<< YOUR CODE HERE >>>
    @_('PRINT LPAREN [ expression ] RPAREN SEMI')
    def print_statement(self, p):
        return (Print(p.expression, coord=self._token_coord(p)))

    # <read_statement> ::= "read" "(" <argument_expression> ")" ";"
    # <<< YOUR CODE HERE >>>
    @_('READ LPAREN argument_expression RPAREN SEMI')
    def read_statement(self, p):
        return (Read(p.argument_expression, coord=self._token_coord(p)))

Complete o código abaixo, escrevendo um método `visit_XXX` para cada tipo de nó AST.

In [ ]:
from os import name
from IPython.core.application import Instance
class Visitor(NodeVisitor):
    '''
    Program visitor class. This class uses the visitor pattern. You need to define methods
    of the form visit_NodeName() for each kind of AST node that you want to process.
    '''
    def __init__(self):
        # Keep a reference to current symbol table
        self.symtab = None

        # Keep a reference to current function
        self.func = None

        # Add built-in type names (int, char, void)
        self.typemap = {
            "int": IntType,
            "char": CharType,
            "string": StringType,
            "bool": BoolType,
            "void": VoidType,
        }
        
    def _assert_semantic(self, condition, msg_code, coord, name="", ltype="", rtype=""):
        """Check condition, if false print selected error message and exit"""
        error_msgs = {
             1: f"'{name}' is not defined",
             2: f"subscript must be of type 'int', not type '{ltype}'",
             3: f"Expression must be of type 'bool', not type '{ltype}'",
             4: f"Cannot assign type '{rtype}' to type '{ltype}'",
             5: f"Assignment operator '{name}' is not supported by type '{ltype}'",
             6: f"Binary operator '{name}' does not have matching LHS/RHS types",
             7: f"Binary operator '{name}' is not supported by type '{ltype}'",
             8: "Break statement must be inside a loop",
             9: f"Array has incomplete element type '{ltype}'",
            10: f"Size mismatch on '{name}' initialization",
            11: f"'{name}' initialization type mismatch",
            12: f"'{name}' initialization must be a single element",
            13: "Lists have different sizes",
            14: "List & variable have different sizes",
            15: f"Variable declared as array of functions of type '{ltype}'",
            16: f"'{name}' is not a function",
            17: f"no. arguments to call '{name}' function mismatch",
            18: f"Type mismatch with parameter '{name}'",
            19: f"The condition expression must be of type(bool), not type '{ltype}'",
            20: "Expression must be a constant",
            21: "Expression is not of basic type",
            22: f"'{name}' does not reference a variable of basic type",
            23: f"'{name}' is not a variable",
            24: f"Return of type '{ltype}' is incompatible with type '{rtype}' function definition",
            25: f"Name '{name}' is already defined in this scope",
            26: f"Unary operator '{name}' is not supported by type '{ltype}'",
            27: f"Conflicting declarations for function '{name}'",
            28: f"'{name}' initialization must be a list or string literal",
            29: "Lists have different types",
            30: "Subscripted value is not an array",
            31: "Expression must be a variable",
            32: "Expression is not assignable",
            33: f"Variable has incomplete type '{ltype}'",
            34: f"'{name}' initialization must be a list",
            35: "Undefined error",
        }
        if not condition:
            msg = error_msgs.get(msg_code)
            print("SemanticError: %s %s" % (msg, coord), file=sys.stdout)
            sys.exit(1)

    def visit_ArrayDecl(self, node):

        # <<< YOUR CODE HERE >>>
        self.visit(node.type)
        self.uc_type = node.type
        self._assert_semantic(isinstance(node.type, FunctionType), 15, node.coord, ltype = node.type)
        if(node.type == ArrayType):
          self._assert_semantic(node.size is not None, 9, node.coord, ltype = node.type)
        if(node.size != None):
          self._assert_semantic(isinstance(node.type,Constant), 20, node.coord) 
          self.visit(node.size)
          self._assert_semantic(node.size.attrs['uc_type'] is IntType, 2, node.coord, ltype = node.type) 

        node.attrs['uc_type'] = ArrayType(node.type.attrs['uc_type'], node.size)

        #conferir isso

        pass

    def visit_ArrayRef(self, node):

        # <<< YOUR CODE HERE >>>

        pass

    def visit_Assert(self, node):

        # <<< YOUR CODE HERE >>>
        self.visit(node.expr)
        self._assert_semantic(node.expr == BoolType, 3, node.coord, ltype = node.expr)
        pass

    def visit_Assignment(self, node):
        # Visit the right side
        self.visit(node.rvalue)
        rtype = node.rvalue.attrs['uc_type']
        # Visit the left side
        self.visit(node.lvalue)        
        ltype = node.lvalue.attrs['uc_type']
        # Check that the assignment is allowed otherwise return a type error (code 4)
        self._assert_semantic(ltype == rtype, 4, node.coord, ltype=ltype, rtype=rtype)
        # Check that assign_ops is supported by the type or return an error (code 5)
        self._assert_semantic(node.op in ltype.assign_ops, 5, node.coord, name=node.op, ltype=ltype)
        # Assign the type of the left side to current node
        node.attrs['uc_type'] = ltype

    def visit_BinaryOp(self,node):
        # Visit the left and right expression
        self.visit(node.left)
        ltype = node.left.attrs['uc_type']
        self.visit(node.right)
        rtype = node.right.attrs['uc_type']
        # TODO: 
        # - Make sure left and right operands have the same type
        # - Make sure the operation is supported
        # - Assign the result type to current node

    def visit_Break(self, node):

        # <<< YOUR CODE HERE >>>
        pass
    def visit_Compound(self, node):

        # <<< YOUR CODE HERE >>>
        tabela = self.symtab
        node.attrs['symtab'] = SymbolTable(self.symtab)
        # Set the reference to current symbol table to the new symbol table
        self.symtab = node.attrs['symtab']
        if(self.func is not None and self.func.args is not None):
          for teste in node.attrs['symtab']:
            node.attrs['symtab'].add(teste,self.func.attrs['symtab'].lookup(teste))
        for _decl in node.decls:
          self.visit(_decl)
        for _stmts in node.stmts:
            self.visit(_stmts)
        self.symtab = tabela
        pass

    def visit_Constant(self, node):

        # <<< YOUR CODE HERE >>>
        node.attrs['uc_type'] =  uCType(Type(node.type))
        pass
        
    def visit_Decl(self, node):

        # <<< YOUR CODE HERE >>>
        self.visit(node.type)
        node.attrs['uc_type'] = node.type.attrs['uc_type']
        if(self.symtab.lookup(node.name) is not None):
          if isinstance(node.type.attrs['uc_type'], FunctionType):
             self._assert_semantic(not node.name.attrs['uc_type'].__eq__(self.symtab.lookup(node.name)), 27, node.coord, name=node.name)
          else:
             self._assert_semantic(not node.name.attrs['uc_type'].__eq__(self.symtab.lookup(node.name)), 25, node.coord, name=node.name)
             self._assert_semantic(node.type != VoidType, 33, node.coord, name=node.name) 
        self.symtab.add(node.name.name, node.name) 
                      
        node.name.attrs['scopes'] = self.symtab
        node.name.attrs['uc_type'] = node.type.attrs['uc_type']
        if isinstance(node.type.attrs['uc_type'], FunctionType):
          for teste in node.type.attrs['uc_type'].params: #checa parametros
           self.symtab.add(teste.name,teste)
        if node.init is not None:
          self._assert_semantic(node.type.attrs['uc_type'] is not FunctionType, 23, node.coord, name=node.init)   
          self.visit(node.init)
            #visito como no roteiro
          if node.type is ArrayType:
              self._assert_semantic(node.init.attrs['uc_type'] is InitList or node.init.attrs['uc_type'] is StringType, 28, node.coord, name=node.name)
          print(node.init.attrs)
          if node.init is InitList or node.init.attrs['uc_type'] is StringType: #conferir isso
              if node.init.attrs['uc_type'] is InitList:
                self._assert_semantic(node.init.attrs['uc_type'] == node.type.attrs['uc_type'], 11, node.coord, node.name)
              if node.type.size is not None:
                self._assert_semantic(node.init.size != node.type.size, 14, node.coord) #confirmar 
              else:
                node.type.size = node.init.size
              if node.init.attrs['uc_type'] is StringType:
                self._assert_semantic(not node.type.type is ArrayType, 34, node.coord, name = node.name) 
                self._assert_semantic(node.type.type.attrs['uc_type'] is CharType, 11, node.coord, name = node.name.name)
              visitante = ConstantVisitor()
              visitante.visit(node.init)   
              visitante = ConstantVisitor()
              visitante.visit(node.init)
              tamanho = 0
              for i in visitante.values:
                tamanho += (len(i) - 2)
              if node.type.size is not None:
                self._assert_semantic(node.type.size != tamanho, 10, node.coord, node.name)
              else:
                node.init.attrs['uc_type'].size = tamanho  
          if node.type.attrs['uc_type'] in (IntType,BoolType,CharType, StringType):
              self._assert_semantic(not(node.init is InitList and node.init.attrs['uc_type'].size > 1), 12, node.coord, node.name) 
          if(node.init.attrs['uc_type'] is StringType):
            self._assert_semantic(node.type.type.attrs['uc_type'] in (CharType, StringType), 4, node.name.coord, ltype = node.type.attrs['uc_type'], rtype = node.init.attrs['uc_type'])          
          else:
            self._assert_semantic(node.type.type.attrs['uc_type'] == node.init.attrs['uc_type'], 4, node.name.coord, ltype = node.type.attrs['uc_type'], rtype = node.init.attrs['uc_type'])
        else:
          if node.type == ArrayType: 
            self._assert_semantic(node.type.size is None, 9, node.coord, ltype = node.type)
        pass
    def visit_DeclList(self, node):

        # <<< YOUR CODE HERE >>>
        node.attrs['symtab'] = SymbolTable(self.symtab)
        for _decl in node.decls:  #copiado da funcao Program
          self.visit(_decl) 
        pass
    def visit_EmptyStatement(self, node):

        # <<< YOUR CODE HERE >>>
        pass
    def visit_ExprList(self, node):

        # <<< YOUR CODE HERE >>>
        for exprs in node.exprs:
          self.visit(exprs)
        pass
    def visit_For(self, node):

        # <<< YOUR CODE HERE >>>
        pass
    def visit_FuncCall(self, node):

        # <<< YOUR CODE HERE >>>
        pass
    def visit_FuncDecl(self, node):

        # <<< YOUR CODE HERE >>>
        self.visit(node.type)
        lista = [] #cria uma lista de nome qualquer
        if node.args != None:
            self.visit(node.args)
            node.attrs['symtab'] = node.args.attrs['symtab']
            for args in node.args.attrs['symtab']:
              lista.append(node.args.attrs['symtab'].lookup(args))
        node.attrs['uc_type'] = FunctionType(node.type.attrs['uc_type'], lista) #COnferir isso que esta bizarro
        pass
    def visit_FuncDef(self, node):

        # <<< YOUR CODE HERE >>>
        lista = []
        node.attrs['loops'] = lista
        self.visit(node.decl)
        self.visit(node.spec)
        referencia = self.func
        self.node = referencia
        self.visit(node.body)
        self.func = referencia
        self.func = node.decl.type
        pass
    def visit_GlobalDecl(self, node):

        # <<< YOUR CODE HERE >>>
        for decls in node.decls:
          self.visit(decls)
        pass
    def visit_ID(self, node): #com problemas

        # <<< YOUR CODE HERE >>>
        if self.symtab.lookup(node.name) is not None:
          self.node.attrs['uc_type'] = self.symtab.lookup(node.name).attr['uc_type']
          self.node.attrs['scope'] = self.symtab.lookup(node.name).attr['scope']
          if isinstance(self.symtab.lookup(node.name), FunctionType):
            self.node.attrs['symtab'] = self.symtab.lookup(node.name).attr['symtab']
            print(self.symtab.lookup(node.name).name)
        else :
          self._assert_semantic(self.symtab.lookup(node.name) is not None, 1 , node.coord, name = node.name)  #conferir 

        pass
    def visit_If(self, node):

        # <<< YOUR CODE HERE >>>
        self.visit(node.cond)
        self._assert_semantic(node.cond == BoolType, 19 , node.coord, ltype = node.cond)
        self.visit(node.ifthen)
        if node.ifelse != None:
          self.visit(node.ifelse)

        pass
    def visit_InitList(self, node):

        # <<< YOUR CODE HERE >>>
        pass
    def visit_ParamList(self, node):

        # <<< YOUR CODE HERE >>>
        pass
    def visit_Print(self, node):

        # <<< YOUR CODE HERE >>>
        pass
    def visit_Program(self, node):
        # Create a symbol table for the file scope and assign it to current node
        node.attrs['symtab'] = SymbolTable()
        # Set the reference to current symbol table to the new symbol table
        self.symtab = node.attrs['symtab']
        # Visit all of the global declarations
        for _decl in node.gdecls:
            self.visit(_decl)

    def visit_Read(self, node):

        # <<< YOUR CODE HERE >>>
        pass
    def visit_Return(self, node):

        # <<< YOUR CODE HERE >>>
        pass
    def visit_Type(self, node):

        # <<< YOUR CODE HERE >>>
        if node.name == 'void':
          node.attrs['uc_type'] = VoidType 
        if node.name == 'bool':
          node.attrs['uc_type'] = BoolType
        if node.name == 'int':
          node.attrs['uc_type'] = IntType 
        if node.name == 'char':
          node.attrs['uc_type'] = CharType
        if node.name == 'string':
          node.attrs['uc_type'] = StringType  
        
        pass
    def visit_UnaryOp(self, node):

        # <<< YOUR CODE HERE >>>
        pass
    def visit_VarDecl(self, node):

        # <<< YOUR CODE HERE >>>
        self.visit(node.type)
        node.attrs['uc_type'] = node.type.attrs['uc_type']
       
        pass
    def visit_While(self, node):

        # <<< YOUR CODE HERE >>>
        
        pass

In [ ]:
def print_error(msg, x, y):
    # use stdout to match with the output in the .out test files
    print("Lexical error: %s at %d:%d" % (msg, x, y), file=sys.stdout)
    
def main(args):
    parser = UCParser(print_error)
    with open(args[0], 'r') if len(args) > 0 else sys.stdin as f:
        ast = parser.parse(f.read())
        if ast is not None:
            sema = Visitor()
            sema.visit(ast)
            ast.show(showcoord=True)

## Teste
Para o desenvolvimento inicial, tente executar o analisador semântico em um arquivo de entrada de exemplo, como:

```cpp
/* comment */
int j = 3;
int main () {
  int i = j;
  int k = 3;
  int p = 2 * j;
  assert p == 2 * i;
}
```

In [ ]:
%%file test.uc 
int main () {
  int i = j;
}

E o resultado será semelhante ao texto mostrado abaixo.

```
Program:
    GlobalDecl:
        Decl: ID(name=j)
            VarDecl:
                Type: int @ 2:1
            Constant: int, 3 @ 2:9
    FuncDef:
        Type: int @ 3:1
        Decl: ID(name=main)
            FuncDecl: @ 3:10
                VarDecl:
                    Type: int @ 3:1
        Compound: @ 3:13
            Decl: ID(name=i)
                VarDecl:
                    Type: int @ 4:3
                ID: j @ 4:11
            Decl: ID(name=k)
                VarDecl:
                    Type: int @ 5:3
                Constant: int, 3 @ 5:11
            Decl: ID(name=p)
                VarDecl:
                    Type: int @ 6:3
                BinaryOp: * @ 6:11
                    Constant: int, 2 @ 6:11
                    ID: j @ 6:15
            Assert: @ 7:3
                BinaryOp: == @ 7:10
                    ID: p @ 7:10
                    BinaryOp: * @ 7:15
                        Constant: int, 2 @ 7:15
                        ID: i @ 7:19
```

In [ ]:
main(["test.uc"])

Estude cuidadosamente a saída do analisador semântico e certifique-se de que ela faz sentido. Quando estiver razoavelmente satisfeito com a saída, tente executar alguns dos testes mais complicados projetados para testar vários cenários atípicos, fora do padrão esperado. Você pode usar como base os exemplos contidos [aqui](https://colab.research.google.com/drive/13o4SdAWIlpDYZ6rU308LqiwXsd0dwRe5?usp=sharing). 

No [AVA](https://ava2.ead.ufscar.br/mod/quiz/view.php?id=532257) há um grande conjunto de testes para verificar seu código: confira-os para ver mais exemplos.

## Envie seu trabalho
Depois de concluir esta tarefa, copie o código da [classe `Visitor`](#scrollTo=ZCLdnLBv5Zz0) e o submeta no [AVA](https://ava2.ead.ufscar.br/mod/quiz/view.php?id=532257).

## Anexo

A lista abaixo define os campos e os atributos que devem ser armazenados em cada tipo de nó AST:

```
###########################################################
## Nós do uC ##############################################
###########################################################

#   <name>*     - um nó filho
#   <name>**    - uma sequência de nós filhos
#   <name>      - um atributo

# 'loop*'       - um nó que declara um laço
# 'loops**'     - uma lista de nós que declaram laços
# 'scope'       - tabela de símbolos que contém o nó
# 'symtab'      - tabela de símbolos de um dado escopo
# 'uc_type'     - um objeto uCType

ArrayDecl: [type*, size*], attrs={'uc_type'}

ArrayRef: [name*, subscript*], attrs={'uc_type'}

Assert: [expr*], attrs={}

Assignment: [op, lvalue*, rvalue*], attrs={'uc_type'}

BinaryOp: [op, left*, right*], attrs={'uc_type'}

Break: [], attrs={'loop*'}

Compound: [decls**, stmts**], attrs={'symtab'}

Constant: [type, value], attrs={'uc_type'}

Decl: [name, type*, init*], attrs={}

DeclList: [decls**], attrs={'symtab'}

EmptyStatement: [], attrs={}

ExprList: [exprs**], attrs={}

For: [init*, cond*, next*, stmt*], attrs={}

FuncCall: [name*, args*], attrs={'uc_type'}

FuncDecl: [args*, type*], attrs={'symtab', 'uc_type'}

FuncDef: [spec*, decl*, body*], attrs={'loops**'}

GlobalDecl: [decls**], attrs={}

ID: [name], attrs={'scope', 'uc_type'} se for variável ou attrs={'symtab', 'scope', 'uc_type'} se for função

If: [cond*, ifthen*, ifelse*], attrs={}

InitList: [exprs**], attrs={'uc_type'}

ParamList: [params**], attrs={'symtab'}

Print: [expr*], attrs={}

Program: [gdecls**], attrs={'symtab'} # OBS: pode ser GlobalDecl ou FuncDef

Read: [expr*], attrs={}

Return: [expr*], attrs={'uc_type'}

Type: [name], attrs={'uc_type'}

UnaryOp: [op, expr*], attrs={'uc_type'}

VarDecl: [declname, type*], attrs={'uc_type'} # OBS: não imprima declname

While: [cond*, stmt*], attrs={}
```